In [1]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [1]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [2]:
os.environ["GROQ_API_KEY"] = "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"

In [40]:
# laoding the document
loader = UnstructuredFileLoader("/content/data/Automotive Service_ Inspection, Maintenance, Repair.pdf")
documents = loader.load()

KeyboardInterrupt: 

In [21]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [22]:
texts = text_splitter.split_documents(documents)

In [23]:
type(texts)

list

In [24]:
len(texts)

3455

In [25]:
#texts[:3]

In [26]:
embeddings = HuggingFaceEmbeddings()

In [27]:
persist_directory = "doc_db"

In [28]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [29]:
# retriever
retriever = vectordb.as_retriever()

In [30]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [31]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [32]:
# invoke the qa chain and get a response for user query
query = "When I am applying brakes, it is making wierd sounds."
response = qa_chain.invoke({"query": query})

In [33]:
response

{'query': 'When I am applying brakes, it is making wierd sounds.',
 'result': "Based on the provided context, the weird sounds you're experiencing when applying brakes could be due to several reasons. Some possible causes include:\n\n1. Squeaks: This could be due to the lining sticking to the rotor momentarily before slipping and then sticking again, causing a high-frequency vibration.\n2. Metal-to-metal sounds: This could be due to excessive wear on the brake pads, causing the metal to rub against the rotor.\n3. Rattles: This could be due to loose parts in the brake system.\n4. Rubbing: This could be due to a distorted backing plate.\n\nIt's also possible that the sounds are caused by vibration, which is a common issue in brake systems.\n\nI would recommend having a professional mechanic inspect your brake system to determine the exact cause of the weird sounds and recommend the necessary repairs.",
 'source_documents': [Document(metadata={'source': '/content/Automotive Service_ Inspe

In [34]:
print(response["result"])

Based on the provided context, the weird sounds you're experiencing when applying brakes could be due to several reasons. Some possible causes include:

1. Squeaks: This could be due to the lining sticking to the rotor momentarily before slipping and then sticking again, causing a high-frequency vibration.
2. Metal-to-metal sounds: This could be due to excessive wear on the brake pads, causing the metal to rub against the rotor.
3. Rattles: This could be due to loose parts in the brake system.
4. Rubbing: This could be due to a distorted backing plate.

It's also possible that the sounds are caused by vibration, which is a common issue in brake systems.

I would recommend having a professional mechanic inspect your brake system to determine the exact cause of the weird sounds and recommend the necessary repairs.


In [35]:
print(response["source_documents"][0].metadata["source"])

/content/Automotive Service_ Inspection, Maintenance, Repair.pdf


In [36]:
# invoke the qa chain and get a response for user query
query = "There was some fluid on my driveway and now my car is not starting. What should I do ?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

Based on the provided context, I don't have enough information to determine the exact cause of your car not starting. However, I can suggest some general steps you can take:

1. Check the battery: Make sure the battery terminals are clean and free of corrosion. If the terminals are corroded, you can try cleaning them with a mixture of baking soda and water.
2. Check the battery electrolyte level: If your battery has removable caps, check the electrolyte level. If the level is low, you can try adding distilled water to the battery.
3. Check the cables: Inspect the battery cables for any signs of damage or corrosion. Make sure the cables are securely connected to the battery terminals.
4. Check the starter motor: If the starter motor is not working, it may be due to a faulty starter motor or a problem with the ignition system.

However, since you mentioned that there was fluid on your driveway, it's possible that the fluid may have leaked from your car and caused the problem. In this cas

In [41]:
# invoke the qa chain and get a response for user query
query = "Is it possible that my alternators are not working?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

Based on the provided context, it's possible that your alternator(s) might not be working properly. However, I would need more information about the symptoms you're experiencing with your vehicle to give a more accurate answer.

Some common signs of a faulty alternator include:

* Low charge rate or no charging at all
* Dim or flickering headlights
* Dead battery
* Engine stalling or difficulty starting
* Unusual noises or vibrations from the alternator

If you're experiencing any of these symptoms, it's possible that your alternator(s) might be malfunctioning. However, it's also possible that the issue could be related to other components in the charging system, such as the voltage regulator, battery, or wiring.

To determine if your alternator(s) are working properly, you may want to perform some tests, such as a voltage drop test on the ground side of the regulator or a recharge current test. However, I would recommend consulting a professional mechanic or a repair manual specific t

In [45]:
!zip -r doc_db.zip /content/doc_db/

  adding: content/doc_db/ (stored 0%)
  adding: content/doc_db/chroma.sqlite3 (deflated 43%)
  adding: content/doc_db/d6e27d78-a0c3-4ae6-9a02-8c24f60f89f3/ (stored 0%)
  adding: content/doc_db/d6e27d78-a0c3-4ae6-9a02-8c24f60f89f3/link_lists.bin (deflated 83%)
  adding: content/doc_db/d6e27d78-a0c3-4ae6-9a02-8c24f60f89f3/length.bin (deflated 46%)
  adding: content/doc_db/d6e27d78-a0c3-4ae6-9a02-8c24f60f89f3/header.bin (deflated 56%)
  adding: content/doc_db/d6e27d78-a0c3-4ae6-9a02-8c24f60f89f3/index_metadata.pickle (deflated 42%)
  adding: content/doc_db/d6e27d78-a0c3-4ae6-9a02-8c24f60f89f3/data_level0.bin (deflated 10%)


In [47]:
# Downloading doc_db
from google.colab import files
files.download('doc_db.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>